# RIPEMD-128 Implementation

A pure Python implementation of the RIPEMD-128 cryptographic hash function.

## Overview

RIPEMD-128 is a cryptographic hash function that produces a 128-bit (16-byte) hash digest. It was developed as part of the RIPEMD family of hash functions and is designed to be a secure alternative to other hash functions of similar output size.

## Features

- **Pure Python Implementation**: No external dependencies required
- **Standard Compliance**: Follows the RIPEMD-128 specification
- **Flexible Input**: Accepts both string and bytes input
- **Hexadecimal Output**: Returns hash as uppercase hexadecimal string

## Functions

### `preprocess(message_bytes)`
Pads the input message according to RIPEMD-128 specifications:
- Appends 0x80 byte
- Pads with zeros to make length ≡ 56 (mod 64)
- Appends original bit length as 64-bit little-endian integer

### `RIPEMD_128_hash(message_str)`
Main hashing function that:
- Accepts string or bytes input
- Processes message in 512-bit blocks
- Uses two parallel computation lines
- Returns 128-bit hash as hexadecimal string

### Helper Functions
- `f(j, x, y, z)`: Round function that varies by round number
- `K(j)`, `k_dash(j)`: Round constants for left and right lines
- `r(j)`, `r_dash(j)`: Message word selection
- `s(j)`, `s_dash(j)`: Rotation amounts
- `rol(x, s)`: Left rotation function

## Algorithm Details

RIPEMD-128 uses:
- **Block size**: 512 bits (64 bytes)
- **Output size**: 128 bits (16 bytes)
- **Rounds**: 64 rounds (2 parallel lines of 64 rounds each)
- **Initial values**: Four 32-bit constants
- **Operations**: XOR, AND, OR, NOT, addition, left rotation

## Example Output

```python
print(RIPEMD_128_hash("RIPEMD 128 is a cryptographic hash function.").upper())
# Output: E2B740FC600A83FF3F1F6EC03545E910
```

## Security Note

RIPEMD-128, while not broken, is considered less secure than longer hash functions like RIPEMD-160 or SHA-256 due to its smaller output size. It should be used only where 128-bit security is sufficient and legacy compatibility is required.

## File Structure

The implementation consists of:
- Message preprocessing function
- Round functions and constants
- Main hashing algorithm
- Utility functions for bit operations

## Compatibility

This implementation is compatible with:
- Python 3.6+
- Any system supporting Python (cross-platform)
- Both string and binary data inputs

In [1]:
def preprocess(message_bytes):
    if(not isinstance(message_bytes, bytes)):
        raise TypeError("Input must be of type bytes")
    m = bytearray(message_bytes)
    bit_length = len(m) * 8

    # Append 0x80 (10000000 in binary)
    m.append(0x80)

    # Compute number of zero bytes to append
    padding_len = (56 - (len(m) % 64)) % 64
    m.extend([0x00] * padding_len)

    # Append original bit length as a 64-bit little-endian integer
    m.extend(bit_length.to_bytes(8, byteorder='little'))

    return m

In [2]:
def f(j,x,y,z):
    if(j>=0 and j<=15):
        return x^ y ^ z
    elif(j>=16 and j<=31):
        return (x & y) | (~x & z)   
    elif(j>=32 and j<=47):
        return (x | ~y) ^ z
    elif(j>=48 and j<=63):
        return (x & z) | (y & ~z)

In [3]:
def K(j):
    if(j>=0 and j<=15):
        return 0x00000000
    elif(j>=16 and j<=31):
        return 0x5A827999
    elif(j>=32 and j<=47):
        return 0x6ED9EBA1
    elif(j>=48 and j<=63):
        return 0x8F1BBCDC
def k_dash(j):
    if(j>=0 and j<=15):
        return 0x50A28BE6
    elif(j>=16 and j<=31):
        return 0x5C4DD124
    elif(j>=32 and j<=47):
        return 0x6D703EF3
    elif(j>=48 and j<=63):
        return 0x00000000

In [4]:
def r(j):
    if(j>=0 and j<=15):
        return j
    elif(j>=16 and j<=31):
        return [7, 4, 13, 1, 10, 6, 15, 3, 12, 0, 9, 5, 2, 14, 11, 8][j-16]
    elif(j>=32 and j<=47):
        return [3, 10, 14, 4, 9, 15, 8, 1, 2, 7, 0, 6, 13, 11, 5, 12][j-32]
    elif(j>=48 and j<=63):
        return [1, 9, 11, 10, 0, 8, 12, 4, 13, 3, 7, 15, 14, 5, 6, 2][j-48]
def r_dash(j):
    if(j>=0 and j<=15):
        return [5, 14, 7, 0, 9, 2, 11, 4, 13, 6, 15, 8, 1, 10, 3, 12][j]
    elif(j>=16 and j<=31):
        return [6, 11, 3, 7, 0, 13, 5, 10, 14, 15, 8, 12, 4, 9, 1, 2][j-16]
    elif(j>=32 and j<=47):
        return [15, 5, 1, 3, 7, 14, 6, 9, 11, 8, 12, 2, 10, 0, 4, 13][j-32]
    elif(j>=48 and j<=63):
        return [8, 6, 4, 1, 3, 11, 15, 0, 5, 12, 2, 13, 9, 7, 10, 14][j-48]

In [5]:
def s(j):
    if(j>=0 and j<=15):
        return [11, 14, 15, 12, 5, 8, 7, 9, 11, 13, 14, 15, 6, 7, 9, 8][j]
    elif(j>=16 and j<=31):
        return [7, 6, 8, 13, 11, 9, 7, 15, 7, 12, 15, 9, 11, 7, 13, 12][j-16]
    elif(j>=32 and j<=47):
        return [11, 13, 6, 7, 14, 9, 13, 15, 14, 8, 13, 6, 5, 12, 7, 5][j-32]
    elif(j>=48 and j<=63):
        return [11,12 ,14 ,15 ,14 ,15 ,9 ,8 ,9 ,14 ,5 ,6 ,8 ,6 ,5 ,12][j-48]
def s_dash(j):
    if(j>=0 and j<=15):
        return [8, 9, 9, 11, 13, 15, 15, 5, 7, 7, 8, 11, 14, 14, 12, 6][j]
    elif(j>=16 and j<=31):
        return [9, 13, 15, 7, 12, 8, 9, 11, 7, 7, 12, 7, 6, 15, 13, 11][j-16]
    elif(j>=32 and j<=47):
        return [9, 7, 15, 11, 8, 6, 6, 14, 12, 13, 5, 14, 13, 13, 7, 5][j-32]
    elif(j>=48 and j<=63):
        return [15 ,5 ,8 ,11 ,14 ,14 ,6 ,14 ,6 ,9 ,12 ,9 ,12 ,5 ,15 ,8][j-48]

In [6]:
h0 = 0x67452301; h1 = 0xEFCDAB89; h2 = 0x98BADCFE; h3 = 0x10325476;

In [7]:
"""
rols denotes cyclic left shift (rotate) over s bit positions.
"""
def rol(x, s):
    return ((x << s) | (x >> (32 - s))) & 0xFFFFFFFF

In [10]:
def RIPEMD_128_hash(message_str):
    # Initial hash values
    h0= 0x67452301
    h1= 0xEFCDAB89
    h2= 0x98BADCFE
    h3= 0x10325476
    if isinstance(message_str, str):
        message = message_str.encode("utf-8")
    else:
        message = message_str
    padded_message = preprocess(message)
    
    # Process the message in successive 512-bit chunks
    for i in range(0, len(padded_message), 64):
        block = padded_message[i:i+64]
        
        # Break chunk into sixteen 32-bit little-endian words
        X = []
        for j in range(16):
            X.append(int.from_bytes(block[j*4:j*4+4], byteorder='little'))
        
        # Initialize hash value for this chunk
        A = h0; B = h1; C = h2; D = h3;
        A_prime = h0; B_prime = h1; C_prime = h2; D_prime = h3; 
        
        # Main loop
        for j in range(64):
            # Left line
            T = (A + f(j, B, C, D) + X[r(j)] + K(j)) & 0xFFFFFFFF
            T = rol(T, s(j))
            A = D
            D = C
            C = B
            B = T
            # Right line
            T_prime = (A_prime + f(63-j, B_prime, C_prime, D_prime) + X[r_dash(j)] + k_dash(j)) & 0xFFFFFFFF
            T_prime = rol(T_prime, s_dash(j))
            A_prime = D_prime
            D_prime = C_prime
            C_prime = B_prime
            B_prime = T_prime
            
        # Combine results
        T = (h1 + C + D_prime) & 0xFFFFFFFF
        h1 = (h2 + D + A_prime) & 0xFFFFFFFF
        h2 = (h3 + A + B_prime) & 0xFFFFFFFF
        h3 = (h0 + B + C_prime) & 0xFFFFFFFF
        h0 = T;
    
    return (h0.to_bytes(4, byteorder='little') + 
            h1.to_bytes(4, byteorder='little') + 
            h2.to_bytes(4, byteorder='little') + 
            h3.to_bytes(4, byteorder='little') ).hex()


In [11]:
print(RIPEMD_128_hash("RIPEMD 128 is a cryptographic hash function.").upper())

E2B740FC600A83FF3F1F6EC03545E910
